In [ ]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master("local")
         .appName("read-postgres")
         # Add postgres jar
         .config("spark.driver.extraClassPath", "/home/tm/work/data/jars/postgresql-42.3.2.jar")
         .getOrCreate())
sc = spark.sparkContext

In [ ]:
df_injuries = (
    spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://postgres:5432/nba")
    .option("dbtable", "public.players_stats")
    .option("user", "nba")
    .option("password", "nba")
    .load()
)